# Create a QGIS Project that Remotely Accesses GIMP Products at NSIDC

## Notebook Purpose

This noteboook allows users to select and search for GIMP products in the NSIDC archive. Once the search is complete, it will build a QGIS project file that allows browsing of the remote data at NSIDC. Once the appropriate steps for authentication have been executed (see **NSIDCLoginNotebook**), the resulting QGIS project can be opened to display products located remotely at NSIDC. The GIMP products are stored as Cloud Optimized Geotiffs (COGs), which allows relatively fast display even over slow internet connections. 

## Setup

There are several python packages that need to be installed to execute this notebook. In addition it may be necessary to install several jupyter lab/notebook extensions. Please follow the procedures outlined in the **NSIDCLoginNotebook**. The instructions there assume an Anaconda (conda) install but should translate well to *pip* or other package managers. The following cell will load all of the necessary packages. If errors occur, make sure all of the packages in the **NSIDCLoginNotebook** are installed. *Note this applies to qgis even if you are running __QGIS__ from a standalone installation*. In principle, the __QGIS__ API should be accessible, but it often hard to find so its often easier to install seperately with conda.

In [ ]:
import glob
import os
import sys
# This path may need altering, or the entire command may be unnecessary.
sys.path.append(os.path.expanduser('~/anaconda3/share/qgis/python'))
import qgis.core as qc
import qgis.gui as qg
from datetime import datetime
import panel as pn
pn.extension()

## Other Code

See **NSIDCLoginNotebook** for more detail on installing the *GIMP* respository (https://github.com/fastice/gimpfunc.git). If the location of the repository is not in your python path, edit the `sys.path.append` argument below to point to the location where the code exists.

In [ ]:
#!git clone https:.... desiredpath/gimpfunc
#sys.path.append('desiredpath/gimpfunc')
import gimpfunc as gimp

## Search for Data

Running the cell below will pull up a panel that allows search for products at NISDC. Products can be filter by date range and subsets of some products can be specified (e.g., velcoity or velocity with errors). Selecting a new product, will append the result to the prior search. Select a product family by pressing the appropriate button (e.g., *NSIDC-0723*), set the filter paramers and date range (default is all dates). Then press *GetData* to execute the search. The search results will be pre-pended to the result from any prior search. To start over, press *ClearSearch* to remove the entire search history. Once the search is complete, the results are saved in `myUrls`, which is used in the steps below. 

In [ ]:
myUrls = gimp.cmrUrls()
myUrls.view()

## Product Hierarchy for QGIS

Products are saved in a QGIS project with the following hierarchy.
* **Category** (optional, e.g., Velocity or Images) 
    * **Product Family1** (e.g., annualVelocity, NSIDC0725)
        * **Year** (optional)
            * **ProductPrefix-YYYY-mm-dd.YYYY-mm-dd (e.g., Vel-2020-01-01.Vel-2020-01-06)**
                * **band1** (e.g., browse, vx, vy)
                * **band2**
                * **...**
    * **Product Family 2**
        * **ProductPrefix-band-YYYY-mm-dd.YYY-mm-dd** (example with only one band, e.g., sigma0)
        * **ProductPrefix-band-YYYY-mm-dd.YYY-mm-dd**
        **...**
        
A `productFamily` represents collection of products of the same type such as annual velocity (NSIDC-0725) mosaics or SAR image mosaics (NSIDC-0723) that are grouped together in QGIS project. Different product families can be established for the same produce line. For example, for the SAR mosaics the **image** products could saved under a product family called images and $\gamma_o$ products could be saved under **gamma0**. Several product families may be grouped under a `Category` such as **Velocity** or **Images**. 

Each product can have a single (e.g,. *v*) or multiple bands (*v*, *vx*, and *vy*). For a single-band product family, the results are saved as `ProductPrefix`-`band`**-YYYY-mm-dd.YYY-mm-dd**. If the product has only a single date, the second date string is ommitted. Multi-band products are grouped together with the name `ProductPrefix`**-YYYY-mm-dd.YYYY-mm-dd**. Each **band** is then saved under its band name (e.g., *vx* and *vy*).

The `Category`, `productFamily`, and `productPrefix` are completely user selectable. The `band` names are those used in the product names, but the user can choose which bands are selected.

By default the products under a product family are organized by year. This feature can be turned off with `byYear=False`. 

The products for each family should all be of the same *type*. Currently supported types are *tif* and *shp*.

An archive search as carried out above returns a list of urls, which may contain data for multiple product families. These products can be seperated appropriately using the the parameters `productFilePrefix` (e.g., *GL_vel_mosaic_Annual* and *GL_S1bks*)and `bands` (e.g., *vx*, *vy*, and *image*) as demonstrated below.      

## Performance Notes

Browsing a single remote product at time can be remarkably fast even with a home connection. But remote access does have its drawbacks. In particular, the time it takes QGIS to load scales with the number of products since the program must verify each product by reading its header information. For 300 products this can take anywhere from 45 seconds to more than 5 minutes. As a result, its not generally a good idea to include too many products in a project. For a partial mitigation of this problem, see working with layer definition files below. Peformance can be improved greatly by downloading the data, at the expense of having to store it locally. See TBD Notebook for searching and downloading data.

Due to network limitations, problems can occur if too many images are selected as visible at once. As a result, the automatically generated *QGIS* project will have most of the layers unchecked at first. When network errors occur, an image will not render properly. If this occurs, uncheck excess images, then zoom in and out, which will trigger a reload and usually will re-render the image correctly (the error flag next to the product may persist even after the image loads correctly).

## Setup and Start a Project

The first step in setting up a QGIS project is to create an instance of `QgisGimpProjectSetup` class as follows.

In [ ]:
myProjectSetup = gimp.QgisGimpProjectSetup(defaultBasePath=None)  # Note since this is remote search, no basePath is required

Examples for image (NSIDC-0723), velocity mosaics (NSIDC-0725, NSIDC-0727, NSIDC-0731), individual glacier TerraSAR-X (NSIDC-0481), and terminus postions (NSIDC-0642) are included below. If the search results from above include the products for a particular product family, they will be included. Otherwise, the QGIS project will contain the Category/productFamily with no data under it. Such 'blanks' can be avoided by selecting the desired categories as follows:

In [ ]:
includeTermini = False # True
includeImages = True #True
includeVelocityMosaics = True
includeTerraSAR = False # True

In [ ]:
QgisProjectFileName = 'qgistest'

Further customization can be performed by modifying the examples below. Note the order of the products in the QGIS legend is determined by the order they are added in the product here (for python $\geq$ 3.7).

### Terminus Positions

After creating `myProjectSetup`, product families need to be defined. The gimp glacier termini products are delivered as shape files (*shp*), which can be added as follows:

In [ ]:
if includeTermini:
    myProjectSetup.addProductFamilies('Termini', productFilePrefix='termini', bands=['termini'], fileType='shp', byYear=False)

### Velocity Mosaics

The following cell will setup product families for the annual, quarterly, and monthly velocity products. Note this step adds an additional level of filtering through `bands`. If a band is selected in the search above (e.g., `vx`) it will not be included unless its explicitly included in with the bands keyword (e.g., `bands=['vx']`).

In [ ]:
if includeVelocityMosaics: # Modify above to include or exclude this step
    # Parameters that apply to all velocity products
    velProperties = {'category': 'Velocity', 'productPrefix': 'Vel',
                     'bands': ['browse', 'vv', 'vx', 'vy'], 'byYear': True, 'fileType': 'tif'}  
    # Create a product family for each mosaic type
    for productFamilyName in ['Annual', 'Quarterly', 'Monthly']:
        myProjectSetup.addProductFamilies(productFamilyName, productFilePrefix=f'GL_vel_mosaic_{productFamilyName}', **velProperties)
    # Modify monthly products
    myProjectSetup.productFamilies['Monthly']['bands'] = ['browse', 'vv']

### Individual Glacier Velocities (TSX)

The following cells will setup a product family for the Selected Glacier Site Velocity Maps from Insar (NSIDC-0481). In the prior example, the default display options were assumped. Those options are are:

In [ ]:
displayOptions = myProjectSetup.defaultDisplayOptions()  # Create a copy of the default options
for key,value in displayOptions.items():
    print(f'{key}: {value}' )

The options in this dictionary can then be modified as follows:

In [ ]:
displayOptions['vv']['colorTable'] = 'Inferno'  # Change color for speed
displayOptions['vv']['maxV'] = 4000  # Change max value at which speed is clipped

The customized options are then passed back along with the other options.

In [ ]:
tsxProperties = {'category': 'TSX', 'productPrefix': 'TSX',
                 'bands': ['vv'], 'byYear': True, 'fileType': 'tif', 
                 'displayOptions': displayOptions}  # Add the display options 

The TSX products are different in that there are many regions as opposed to full Greenland mosaics like the products decribed above. Thus, it would be nice to organize them each under their respective 'box' (e.g., E61.10N - see Figure on Page 4 of the User Guide at https://nsidc.org/data/nsidc-0481). The names of the boxes found by the search can be recovered using `myUrls.findTSXBoxes()`, which is used to produce a product family for each box.

In [ ]:
if includeTerraSAR:
    for productFamilyName in myUrls.findTSXBoxes():  # Note findTSXboxes can be replaced with a list of boxes (e.g., ['W72.90N',...])
        myProjectSetup.addProductFamilies(productFamilyName, productFilePrefix=f'TSX_{productFamilyName}', **tsxProperties)

### Image Mosaics

This example sets up a product family for image products (NSIDC-0723). The code can be edited to change which bands are selected (*image*, *sigma0*, and *gamma0*). As originally written, it includes only *image*  bands (any *sigma0* and *gamma0* products returned by the search will be ignored).

In [ ]:
productFamilyName = 'Image Mosaics'
if includeImages:  # Modify above to include or exclude this step
    myProjectSetup.addProductFamilies(productFamilyName, category='Imagery', productPrefix='SAR', 
                                      productFilePrefix='GL_S1bks', bands=['image'], byYear=True, fileType='.tif')

In the QGIS project, the image products will be organized under the Category **Imagery**, the Product Family **Image Mosaics** and the year of aquisition. Each product will be named **SAR-image-YYYY-mm-dd.YYYY-mm-dd**.

### Adding the Data

In the search process above, a set of urls (links) to the data were collected and stored in `myUrls`. The subsequent steps defined how these products shoud be organized in the *QGIS* project. The next step is to link the products pointed to by the *urls* to the `myProjectSetup`. This step is performed separately for *tif* (aka cog) and shp file products as follows:

In [ ]:
if includeImages or includeVelocityMosaics or includeTerraSAR:
    myProjectSetup.getProductFamilies(urls=myUrls.getCogs())  # Tif products requested so add cogs
if includeTermini:
    myProjectSetup.getProductFamilies(urls=myUrls.getShapes())  # Add the shps if requested

## Build the QGIS Project

The first step is start up the stand-alone version of QGIS core routines as: 

In [ ]:
qgs = qc.QgsApplication([], False)
qgs.initQgis()

Then create a `gimp.QgisProject` object, which will take the setup information from `myProjectSetup` to create `myProject`, which will use it to automatically generate a *QGIS* project.

In [ ]:
myProject = gimp.QgisGimpProject(myProjectSetup)

Layer definition files allow the individual Categories or ProductFamilies to be included in pre-existing products. Or if a project takes too long to load because it has too many layers, layer definition files allow collections of layers to be added or removed as needed (in QGIS **Layers->Add From Layer Definition File**). 

In [ ]:
myProject.saveLayerDefinitions(QgisProjectFileName, saveCategories=True)

In [ ]:
myProject.saveProject(QgisProjectFileName)
qgs.exitQgis()
qgs.exit()

If all went well, this process produced a working QGIS project. In order for it to work, authentication via a valid *NASA Earth Data* is needed to access the data at *NSIDC*. This can be done following the procedures in the **NSIDCLoginNotebook.ipynb** (https://github.com/fastice/GIMPNotebooks).